<a href="https://www.kaggle.com/code/yasstake/kaggle-baseline?scriptVersionId=124138270" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 趣旨

はじめてなのに思いつくままに始めてみたら、時間を大量に消費して行き詰ってしまった。体系的なアプローチが必要そうだ。

そこで「Kaggleで磨く機械学習の実践力（諸橋政幸　著）」のステップに従いやってみることにした。この本はTitanicを例にStepByStepでやり方を書いてあるハンズオン形式の本。ちょうど今回の例もTitanicと同じ２値問題なので応用できそう。

本の章毎にStepByStepで今回の課題をやってみる(具体的には４章から始まる)

著作権の問題もあるので章の名前をのみを引用しています。
そのため全体の考え方などは本をみないとわからないかもしれません。この本、初心者の私にとってはとてもわかりやすかったです。ぜひ、章番号とあわせて読んでみることおすすめ。

# 4 ベースラインの作成

* lightgbmとk-fold法を用いたシンプルなベースラインを作成する
* 特徴量エンジニアリングは最初はやらない。
  * 数値データは全部利用する。
  * 文字列データは一旦削除する（ベースライン完了後の特徴量エンジニアリングで実施する[予定]）

## 4.2分析設計

課題が何であるかを明確にする。
* 目的変数: カラム['target_label']の値
* 目的変数の特徴:「1＝重症」、「0＝重症ではない」の２値
* 評価指標: AUC(Area Under the Curve)

## 4.3 ファイルの読み込み

* 最初に必要ライブラリをインポートする
* 次にデータファイルを読み込む。データファイルは以下の２つ
  * `test_df.csv` テスト（提出用）データ
  * `train_df.csv`　訓練用データ

In [1]:
! pip install numpy
! pip install sklearn
! pip install pandas
! pip install matplotlib


  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=0f6477467af32ffa4dfe77d17a2a19a61533398eeb6914a606d1357a750eee08
  Stored in directory: /root/.cache/pip/wheels/15/9a/b1/2478e73a520d596fab614693f5cd1beef4ba3db737bed1ac7d
Successfully built sklearn


In [2]:
# 必要ファイルの読み込み
import numpy as np
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
# データの読み込み

# for local
#DATA_DIR = './input/'

# for kaggle notebook
DATA_DIR = '/kaggle/input/prediction-of-seriously-ill-patients/'

test_df = pd.read_csv(DATA_DIR + 'test_df.csv')
train_df = pd.read_csv(DATA_DIR + 'train_df.csv')


## 4.4 データの確認（簡易）

In [4]:
# レコード数とカラム数の確認

print('train_df.shape: ', train_df.shape)
print('test_df.shape: ', test_df.shape)

train_df.shape:  (51359, 85)
test_df.shape:  (12840, 84)


### 4.4.2 カラムごとのデータの種類の確認

In [5]:
# show first 5 rows
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
print(train_df.head())

   id  personal_id_1  personal_id_2  facility_id   age        bmi  \
0   0         114501          58009           51  69.0  24.731460   
1   1          44353         112590           19  64.0  28.666129   
2   2           8023           1677           16  74.0  18.144869   
3   3         106340          74166          188  60.0  23.047667   
4   4         118467          52717          168  75.0  20.190265   

   situation_1  situation_2  ethnicity gender  height  weight  icu_id  \
0            0          1.0  Caucasian      M  175.30    76.0     698   
1            0          1.0  Caucasian      M  183.00    96.0     657   
2            0          0.0  Caucasian      F  166.00    50.0     482   
3            0          0.0  Caucasian      M  182.90    77.1     855   
4            0          0.0  Caucasian      F  160.02    51.7     136   

                  icu_1  icu_2         icu_3      icu_4  icu_5   icu_6  icu_7  \
0                 Floor  admit          MICU  25.801389  302.0  1

In [6]:
# show each data types and non null count
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51359 entries, 0 to 51358
Data columns (total 85 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    51359 non-null  int64  
 1   personal_id_1         51359 non-null  int64  
 2   personal_id_2         51359 non-null  int64  
 3   facility_id           51359 non-null  int64  
 4   age                   48987 non-null  float64
 5   bmi                   49490 non-null  float64
 6   situation_1           51359 non-null  int64  
 7   situation_2           50959 non-null  float64
 8   ethnicity             50550 non-null  object 
 9   gender                51346 non-null  object 
 10  height                50645 non-null  float64
 11  weight                49880 non-null  float64
 12  icu_id                51359 non-null  int64  
 13  icu_1                 51299 non-null  object 
 14  icu_2                 51359 non-null  object 
 15  icu_3              

In [7]:
def get_object_column_name(df):
    '''
    returns object columns in dataframe
    '''
    return df.select_dtypes(include=['object']).columns

def strip_object_columns(df):
    '''
    drop columns which has object type
    '''
    return df.select_dtypes(exclude=['object'])
    


print('train_df object columns: ', get_object_column_name(train_df))
print('train_df not object columns: ', strip_object_columns(train_df).columns)


train_df object columns:  Index(['ethnicity', 'gender', 'icu_1', 'icu_2', 'icu_3', 'body_system_1',
       'body_system_2'],
      dtype='object')
train_df not object columns:  Index(['id', 'personal_id_1', 'personal_id_2', 'facility_id', 'age', 'bmi',
       'situation_1', 'situation_2', 'height', 'weight', 'icu_id', 'icu_4',
       'icu_5', 'icu_6', 'icu_7', 'icu_8', 'glasgow_coma_scale_1',
       'glasgow_coma_scale_2', 'glasgow_coma_scale_3', 'glasgow_coma_scale_4',
       'heart_rate', 'blood_oxy', 'arterial_pressure', 'respiratory_rate',
       'temp', 'blood_pressure_1', 'blood_pressure_2', 'blood_pressure_3',
       'blood_pressure_4', 'v1_heartrate_max', 'v2', 'v3', 'v4', 'v5', 'v6',
       'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'w1',
       'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w10', 'w11', 'w12',
       'w13', 'w14', 'w15', 'w16', 'w17', 'w18', 'x1', 'x2', 'x3', 'x4', 'x5',
       'x6', 'aids', 'cirrhosis', 'diabetes', 'hepatic_issue',


### 4.4.3 欠損値の確認

In [8]:
train_df.isnull().sum()

id                         0
personal_id_1              0
personal_id_2              0
facility_id                0
age                     2372
bmi                     1869
situation_1                0
situation_2              400
ethnicity                809
gender                    13
height                   714
weight                  1479
icu_id                     0
icu_1                     60
icu_2                      0
icu_3                      0
icu_4                      0
icu_5                    938
icu_6                    636
icu_7                      0
icu_8                    400
glasgow_coma_scale_1    1086
glasgow_coma_scale_2    1086
glasgow_coma_scale_3     581
glasgow_coma_scale_4    1086
heart_rate               491
blood_oxy                400
arterial_pressure        566
respiratory_rate         709
temp                    2313
blood_pressure_1          87
blood_pressure_2          87
blood_pressure_3         583
blood_pressure_4         583
v1_heartrate_m

# 4.5 データセットの作成

In [9]:
x_train, y_train, id_train = train_df.drop(['id', 'target_label'], axis=1), train_df['target_label'], train_df['id']
x_test, id_test = test_df.drop(['id'], axis=1), test_df['id']

In [10]:
print("x_train:{} | y_train: {} | id_train: {}".format(x_train.shape, y_train.shape, id_train.shape))
print("x_test:{} | id_test: {}".format(x_test.shape, id_test.shape))

x_train:(51359, 83) | y_train: (51359,) | id_train: (51359,)
x_test:(12840, 83) | id_test: (12840,)


# 4.6 バリデーション設計

## 4.6.2 ホールドアウト検証と交差検証

クロスバリデーション（交差検証）を採用して実装してみる。

```
`StratifiedKFold`は、クラスの分布がバランスよくなるように、データを分割する交差検証手法の一種です。具体的には、データセットをk個のサブセットに分割し、各サブセットがクラスの比率を保ったままランダムに抽出されるようにします。これにより、モデルの性能を評価するためのデータが、すべてのクラスにわたって均等に分散することが保証されます。
```

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html

*Params*
 * `n_splits` 分割数
 * `shuffle` バッチ毎にシャッフルするか？（default False)
 * `random_state` クラス毎のシャッフル設定　https://scikit-learn.org/stable/modules/cross_validation.html#stratified-k-fold
 

In [11]:
from sklearn.model_selection import StratifiedKFold

def get_fold(n_splits=5, random_state=1):
    return StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state).split(x_train, y_train)


# データがバランスしているか確認（数・割合）
for i_fold, (train_index, validate_index) in enumerate(get_fold()):
    xf_train, xf_validate = x_train.iloc[train_index], x_train.iloc[validate_index]
    yf_train, yf_validate = y_train.iloc[train_index], y_train.iloc[validate_index]
    
    print('#{}| xf_train: {} | yf_train: {} (serious: {}) | xf_validate: {} | yf_validate: {} (serious: {})'
          .format(i_fold, xf_train.shape, yf_train.shape, yf_train.mean(),
                  xf_validate.shape, yf_validate.shape, yf_validate.mean())
    )



#0| xf_train: (41087, 83) | yf_train: (41087,) (serious: 0.08630467057706817) | xf_validate: (10272, 83) | yf_validate: (10272,) (serious: 0.08625389408099689)
#1| xf_train: (41087, 83) | yf_train: (41087,) (serious: 0.08630467057706817) | xf_validate: (10272, 83) | yf_validate: (10272,) (serious: 0.08625389408099689)
#2| xf_train: (41087, 83) | yf_train: (41087,) (serious: 0.08628033197848468) | xf_validate: (10272, 83) | yf_validate: (10272,) (serious: 0.086351246105919)
#3| xf_train: (41087, 83) | yf_train: (41087,) (serious: 0.08628033197848468) | xf_validate: (10272, 83) | yf_validate: (10272,) (serious: 0.086351246105919)
#4| xf_train: (41088, 83) | yf_train: (41088,) (serious: 0.08630257009345794) | xf_validate: (10271, 83) | yf_validate: (10271,) (serious: 0.08626229188978678)


In [12]:
param_fixed = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.01,
    'num_leaves': 2**5,
    'n_estimators': 10000,
    'random_state': 1,
    'importance_type': 'gain',
}

def train_lgbt(x_train, y_train, x_validate, y_validate, param_fixed):
    import lightgbm as lgb
    from sklearn.metrics import roc_auc_score
    from lightgbm import early_stopping
    
    param = param_fixed.copy()
    
    model = lgb.LGBMClassifier(**param)
    model.fit(strip_object_columns(x_train), y_train, eval_set=[(strip_object_columns(x_validate), y_validate)], callbacks=[early_stopping(100, verbose=False)])
    y_pred = model.predict_proba(strip_object_columns(x_validate))[:, 1]
    auc = roc_auc_score(y_validate, y_pred)
    
    return model, auc



In [13]:
# execute cross validation

def train_fold():
    aucs = []
    models = []

    for i_fold, (train_index, validate_index) in enumerate(get_fold()):
        xf_train, xf_validate = x_train.iloc[train_index], x_train.iloc[validate_index]
        yf_train, yf_validate = y_train.iloc[train_index], y_train.iloc[validate_index]
    
        model, auc_validate = train_lgbt(xf_train, yf_train, xf_validate, yf_validate, param_fixed)
    
        print('\tFold#{}  AUC validate: {}'.format(i_fold, auc_validate))
        
        aucs.append(auc_validate)
        models.append(model)
        
    return np.array(aucs), models

aucs, models = train_fold()

	Fold#0  AUC validate: 0.8994060362703398
	Fold#1  AUC validate: 0.8862291420053594
	Fold#2  AUC validate: 0.8935828539749258
	Fold#3  AUC validate: 0.8773946047177638
	Fold#4  AUC validate: 0.8976511435206509


## 4.7.2 クロスバリデーションの場合

説明変数の重要度の算出を行う。k個あるモデルにおける寄与度の平均をとる。

In [14]:
def display_importance(models):
    importances = pd.DataFrame()
    for i, model in enumerate(models):
        importances['importance_{}'.format(i)] = model.feature_importances_
    importances['importance'] = importances.mean(axis=1)
    importances['feature'] = strip_object_columns(x_train).columns
    importances = importances.sort_values('importance', ascending=False)
    print(importances[['feature', 'importance']])
    
print('AUC mean: {}'.format(aucs.mean()))
display_importance(models)

AUC mean: 0.8908527560978079
                 feature     importance
66                    x5  240460.283288
67                    x6   56045.743843
37                   v10   21890.855690
29                    v2   21086.424623
39                   v12   17902.440419
42                   v15   14442.405769
3                    age   13357.944539
63                    x2   13011.530166
65                    x4   11842.549067
43                   v16   11187.039676
35                    v8   11133.247389
4                    bmi    9622.758914
10                 icu_4    9278.613364
9                 icu_id    9269.743513
12                 icu_6    9164.572711
64                    x3    8787.565539
28      v1_heartrate_max    8782.940390
6            situation_2    7563.765098
34                    v7    7315.132988
23                  temp    7281.518440
62                    x1    7225.767706
55                   w12    6932.469099
41                   v14    6796.022371
31         

## 4.8　モデル推論

k-fold法では、２つの方法がとれるようだ。
1. k-foldでトレーニングしたモデルk個のアンサンブルをとる。
2. k-foldでは、特徴量・ハイパーパラメータの有効性の確認を行い、同じパラメータで全データでトレーニングする。

ここでは１の方法で一番簡単と思われる平均値をとって推論を行う。

In [15]:
def predict(models, x_test):
    y_pred = np.zeros(len(x_test))
    for model in models:
        y_pred += model.predict_proba(strip_object_columns(x_test))[:, 1]
    y_pred /= len(models)
    return y_pred

pred = pd.DataFrame(predict(models, x_test), columns=['target_label'])
pred['id'] = id_test


def create_submission(df):
    FILE_NAME='submission.csv'
    df.to_csv(FILE_NAME, index=False, header=True)
    print('Write complete to [{}]'.format(FILE_NAME))
    
create_submission(pred)


Write complete to [submission.csv]
